In [1]:
using Distributed
if nprocs() < 2
    addprocs(3)
end
using JLD2
using LarSurf
using SparseArrays

┌ Info: Recompiling stale cache file C:\Users\Jirik\.julia\compiled\v1.1\LarSurf\WDzsQ.ji for LarSurf [39bcc72a-2642-51fe-8404-ebe33c8ec443]
└ @ Base loading.jl:1184
[ Info: Loaded M on 1


In [2]:
using SharedArrays

a = SharedArray{Float64}(10)
@sync @distributed for i = 1:10
    a[i] = i
end
a

10-element SharedArray{Float64,1}:
  1.0
  2.0
  3.0
  4.0
  5.0
  6.0
  7.0
  8.0
  9.0
 10.0

In [3]:
function get_EV_quads(FV::Array{Array{Int64,1},1}; return_unfiltered=false, data=nothing)
	@debug "getFV " FV
	# EV = []
	# for f in FV
	# 	push!(EV, [[f[1],f[2]],[f[3],f[4]],  [f[1],f[3]],[f[2],f[4]]])
	# 	# push!(EV, [[f[1],f[2]],[f[3],f[4]],  [f[1],f[3]],[f[2],f[4]]])
	# end
	if size(FV[1],1) == 4

		# couples = [[1,2], [3,4], [1,3], [2,4]]
		couples = [[1,2], [2,3], [3,4], [4,1]]
	else
		couples = [[1,2], [2,3], [3,1]]
	end
	szc = size(couples,1)

	ev_time = @elapsed EV = reshape([sort([f[couples[c][1]], f[couples[c][2]]]) for f in FV, c=1:szc],:)
	# doubleedges = Base.sort(mycat(EV))
	if return_unfiltered
		if data != nothing
			data["smoothing EV construction time [s]"] = ev_time
		end
		return EV
	end
    ev_unique_time = @elapsed EVnew = collect(Set(EV))
	if data != nothing
		data["smoothing EV construction time [s]"] = ev_time
		data["smoothing make EV unique time [s]"] = ev_unique_time
	end
	return EVnew
	# doubleedges = Base.sort(EV)
	# doubleedges = convert(LarSurf.Lar.Cells, doubleedges)
	# newEV = [doubleedges[k] for k=1:2:length(doubleedges)]
	# return newEV
end


get_EV_quads (generic function with 1 method)

In [4]:
function setIzero!(arr)
    for i=1:size(arr)[1]
        arr[i,i] = 0
    end
    return arr
end

function setIzero_p!(arr)
    @distributed for i=1:size(arr)[1]
        arr[i,i] = 0
    end
    return arr
end

function getDiag(arr)
    sz = size(arr)[1]
    arro = zeros(eltype(arr), sz)
    for i=1:sz
        arro[i] = arr[i,i]
    end
    return arro
end

function getDiag_p(arr)
    sz = size(arr)[1]
    arro = zeros(eltype(arr), sz)
    for i=1:sz
        arro[i] = arr[i,i]
    end
    return arro
end



getDiag_p (generic function with 1 method)

In [5]:
a = SharedArray{Float64}(1000,1000)
a = a .+ 1.0;

In [6]:
@elapsed setIzero!(a)

0.010817

In [7]:
@elapsed setIzero_p!(a)

0.017626799

# Smoothing

Data used in next experiment are produced by:
```
cd LarSurf
julia experiments\surface_extraction_parallel.jl -d medical/orig/3Dircadb1.1/MASKS_DICOM/LIVER --show --taubin_n 40 --taubin_lambda 0.5 --taubin_mu -0.2 --n_procs 4 --threshold 0 --color 1. 0. 0. 0.
```

In [72]:
# produced by experiment
#julia experiments\surface_extraction_parallel.jl -d medical/orig/3Dircadb1.1/MASKS_DICOM/LIVER --show --taubin_n 40 --taubin_lambda 0.5 --taubin_mu -0.2 --n_procs 4 --threshold 0 --color 1. 0. 0. 0.
@JLD2.load "../data_V_FV.jld2"

2-element Array{Symbol,1}:
 :V 
 :FV

In [9]:
k1=0.35; k2=-0.1; n=1

1

In [10]:
function smoothing_EV(V, EVch::SparseMatrixCSC, k=0.35)
#     Matrix(setIzero!(EVch' * EVch))

	EVchPow = EVch' * EVch
    neighboorNumber = getDiag(EVchPow)
    neighboors = setIzero!(EVchPow)
	dropzeros!(neighboors)

#     targetV =  neighboors * V' ./ neighboorNumber
#     diff = targetV - V'
#     newV = (V' + k * diff)'

    targetV = (V * neighboors) ./ neighboorNumber'
    # println("targetV shape: ", size(targetV))
    diff = targetV - V
	# @info "sm V orig   $(V[:,1:5])"
	# @info "sm V target $(targetV[:,1:5])"
	# @info "sm V diff   $(diff[:,1:5])"
    # println("diff shape: ", size(diff))
    newV = V + k * diff

    return make_full(newV)
end


smoothing_EV (generic function with 2 methods)

In [11]:
methods(LarSurf.ll2array)

# 1 method for generic function "ll2array":
[1] ll2array(CVill::Array) in LarSurf at C:\Users\Jirik\.julia\dev\LarSurf\src\arr_fcn.jl:274

In [12]:
t_ev = @elapsed EV = get_EV_quads(FV, data=nothing)
println("t_ev $t_ev")

# LarSurf
t_ll2arr = @elapsed aEV = LarSurf.ll2array(EV)
println("t ll2arr $t_ll2arr")

# kEV = LarSurf.characteristicMatrix(aEV, size(bigV)[2])
# t_chm = @elapsed kEV = LarSurf.characteristicMatrix(aEV, size(V)[2])
t_chm = @elapsed kEV = LarSurf.characteristicMatrix_for_loop(aEV, size(V)[2])
println("t charmat $t_chm")
# kEV = Lar.characteristicMatrix(EV)
# @info "computing new V"
newV = V
# taubin_time = @elapsed for i=1:n
# @info "taubin iteration $(i)"
# newV = smoothing_EV(newV, kEV, k1)
# newV = smoothing_EV(newV, kEV, k2)
# end

t_ev 2.402672201
t ll2arr 0.3394426
t charmat 145.9485287


3×272726 Array{Float64,2}:
 0.11705   0.088958  0.266874  0.056184  …  0.440108  0.032774  0.309012
 0.14046   0.449472  0.04682   0.341786     0.346468  0.07023   0.379242
 0.337104  0.28092   0.379242  0.2341       0.238782  0.290284  0.060866

# Slowest is characteristicMatrix

In [13]:
methods(LarSurf.characteristicMatrix)

# 5 methods for generic function "characteristicMatrix":
[1] characteristicMatrix(FV::Array{Array{Int64,1},1}) in LarSurf at C:\Users\Jirik\.julia\dev\LarSurf\src\arr_fcn.jl:185
[2] characteristicMatrix(FV::Array{Int64,2}) in LarSurf at C:\Users\Jirik\.julia\dev\LarSurf\src\arr_fcn.jl:184
[3] characteristicMatrix(FV::Array{Int64,2}, nvertices) in LarSurf at C:\Users\Jirik\.julia\dev\LarSurf\src\arr_fcn.jl:188
[4] characteristicMatrix(CVi, nvertices) in LarSurf at C:\Users\Jirik\.julia\dev\LarSurf\src\arr_fcn.jl:243
[5] characteristicMatrix(CVi, nvertices, axis) in LarSurf at C:\Users\Jirik\.julia\dev\LarSurf\src\arr_fcn.jl:243

In [14]:
function characteristicMatrix( FV::LarSurf.Lar.Cells )::LarSurf.Lar.ChainOp
    I,J,V = Int64[],Int64[],Int8[]
    for f=1:length(FV)
        for k in FV[f]
        push!(I,f)
        push!(J,k)
        push!(V,1)
        end
    end
    M_2 = sparse(I,J,V)
    return M_2
end


characteristicMatrix (generic function with 1 method)

In [15]:
function unsafe_resize(sp::SparseMatrixCSC,m,n)
  newcolptr = sp.colptr
  resize!(newcolptr,n+1)
  for i=sp.n+2:n+1
    newcolptr[i] = sp.colptr[sp.n+1] 
  end
  return SparseMatrixCSC(m,n,newcolptr,sp.rowval,sp.nzval)
end

unsafe_resize (generic function with 1 method)

In [63]:
using SparseArrays

"""
Original implementation
"""
function characteristicMatrix_push( FV::LarSurf.Lar.Cells)::LarSurf.Lar.ChainOp
	I,J,V = Int64[],Int64[],Int8[]
#     println("sz ", size(FV))
#     println("len ", length(FV))
	for f=1:length(FV)
		for k in FV[f]
		push!(I,f)
		push!(J,k)
		push!(V,1)
		end
	end
	M_2 = sparse(I,J,V)
	return M_2
end

function characteristicMatrix_set( FV::LarSurf.Lar.Cells)::LarSurf.Lar.ChainOp
#     println("list of lists  set")
    sz = length(FV) * length(FV[1])
#     print("size $sz")
    I = Array{Int64,1}(undef, sz) 
    J = Array{Int64,1}(undef, sz) 
    V = Array{Int8,1}(undef, sz) 
# 	I,J,V = Int64[],Int64[],Int8[]
#     println("sz ", size(FV))
#     println("len ", length(FV))
    i = 1
	for f=1:length(FV)
		for k in FV[f]
            I[i] = f
            J[i] = k
            V[i] = 1
            i = i + 1
            
# 		push!(I,f)
# 		push!(J,k)
# 		push!(V,1)
		end
	end
	M_2 = sparse(I,J,V)
	return M_2
end

function characteristicMatrix_push( FV::Array{Int64,2})
    I,J,V = Int64[],Int64[],Int8[]
#     println(size(FV))
#     println(length(FV))
    for f=1:size(FV, 1)
        for k in FV[f, :]
        push!(I,f)
        push!(J,k)
        push!(V,1)
        end
    end
    M_2 = sparse(I,J,V)
    return M_2
end


function characteristicMatrix_set( FV::Array{Int64,2})
#     println("chM set")
#     sz = size(FV, 1)
    sz = length(FV)
    I = Array{Int64,1}(undef, sz) 
    J = Array{Int64,1}(undef, sz) 
    V = Array{Int8,1}(undef, sz) 
#     I,J,V = Int64[],Int64[],Int8[]
#     println(size(FV))
#     println(length(FV))
    i = 1
    for f=1:size(FV, 1)
        for k in FV[f, :]
            I[i] = f
            J[i] = k
            V[i] = 1
            i = i + 1
        
#         push!(I,f)
#         push!(J,k)
#         push!(V,1)
        end
    end
#     println("i=$i")
    M_2 = sparse(I,J,V)
    return M_2
end

function characteristicMatrix_parallel( FV::Array{Int64,2})
#     sz = size(FV, 1)
    sz = length(FV)
    I = SharedArray{Int64,1}(sz) 
    J = SharedArray{Int64,1}(sz) 
    V = SharedArray{Int8,1}(sz) 
    ii = SharedArray{Int64,1}(1) 
#     I,J,V = Int64[],Int64[],Int8[]
    println("sz $sz")
    println(size(FV))
    println(length(FV))
    ii[1] = 1
    @sync @distributed for f=1:size(FV, 1)
#     for f=1:size(FV, 1)
        for k in FV[f, :]
            I[ii[1]] = f
            J[ii[1]] = k
            V[ii[1]] = 1
            ii[1] = ii[1] + 1
        
#         push!(I,f)
#         push!(J,k)
#         push!(V,1)
        end
    end
    println("i=${ii[1]}")
    M_2 = sparse(I,J,V)
    return M_2
end

function characteristicMatrix_push(FV::Array{Int64,2}, nvertices)
    mat = characteristicMatrix_push(FV)
    return unsafe_resize(mat, size(mat, 1), nvertices)
end

function characteristicMatrix_set(FV::Array{Int64,2}, nvertices)
    mat = characteristicMatrix_set(FV)
    return unsafe_resize(mat, size(mat, 1), nvertices)
end

LoadError: syntax: invalid interpolation syntax: "${"

In [17]:
print(size(V))
print(size(aEV))
print(size(EV))

(3, 272726)(543517, 2)(543517,)

In [18]:
aEV[1,:]

2-element Array{Int64,1}:
  66752
 218961

In [71]:
println( @elapsed mat = characteristicMatrix_push(aEV))
println( @elapsed mat = characteristicMatrix_set(aEV))
println( @elapsed mat = characteristicMatrix_push(EV))
println( @elapsed mat = characteristicMatrix_set(EV))
# println( @elapsed mat = characteristicMatrix_set(EV))
# println( @elapsed mat = characteristicMatrix_parallel(aEV))
# print(size(mat))

0.258880599
0.088527201
0.1423089
0.086991901


In [20]:
characteristicMatrix(FV::Array{Int64,2}) = characteristicMatrix_push(FV::Array{Int64,2})

characteristicMatrix (generic function with 2 methods)

In [21]:
methods(characteristicMatrix)

# 2 methods for generic function "characteristicMatrix":
[1] characteristicMatrix(FV::Array{Int64,2}) in Main at In[20]:1
[2] characteristicMatrix(FV::Array{Array{Int64,1},1}) in Main at In[14]:2

### Detailed parallel

In [22]:
FV = aEV
sz = length(FV)

I = SharedArray{Int64,1}(sz) 
J = SharedArray{Int64,1}(sz) 
V = SharedArray{Int8,1}(sz) 
ii = SharedArray{Int64,1}(1) 
# @everywhere a = 1
ii[1] = 1
@sync @distributed for f=1:size(FV, 1)
# for f=1:size(FV, 1)
    for k in FV[f, :]
        I[ii[1]] = f
        J[ii[1]] = k
        V[ii[1]] = 1
        ii[1] = ii[1] + 1
#         a = a + 1

    end
#     println("ooi ", ii[1], " ", a)
end
I[10]

5

In [23]:
# sparse(I, J, V)

## Verify outputs

In [24]:
hexV, hexFVarr = LarSurf.hexagon()
# hexFVll = LarSurf.arrayofarray2arrayd2d(hexFVarr)
# hexFVll

([0.0 0.0; 2.0 0.0; … ; 3.0 -2.0; 1.0 -2.0], [1 2 3; 2 4 3; … ; 2 7 6; 2 1 7])

In [42]:
function array2ll(CVarr::Union{Array{Int64,2}, Array{Int8,2}})
    na, nb = size(CVarr)
    arr = Array{Array{eltype(CVarr),1},1}(undef, na)
    for i=1:na
        arr[i] = CVarr[i, :]
    end
    return arr
end
hexFVll = array2ll(hexFVarr)

6-element Array{Array{Int64,1},1}:
 [1, 2, 3]
 [2, 4, 3]
 [4, 2, 5]
 [5, 2, 6]
 [2, 7, 6]
 [2, 1, 7]

In [67]:
println( @elapsed mat0 = LarSurf.characteristicMatrix_for_loop(hexFVarr, size(hexV, 1)))
println( @elapsed mat1 = characteristicMatrix_push(hexFVll ))
println( @elapsed mat2 = characteristicMatrix_set( hexFVll ))
println( @elapsed mat3 = characteristicMatrix_push(hexFVarr))
println( @elapsed mat4 = characteristicMatrix_set( hexFVarr))

2.11e-5
1.4e-5
7.3e-6
8.3e-6
6.7e-6


In [60]:
function match_arr(arr1, arr2)
    for i =1:length(arr1)
        if arr1[i] != arr2[i]
            println("Element i=$i is not in match. $(arr1[i]) != $(arr2[i])")
            return false
        end
    end
    return true
end

match_arr (generic function with 1 method)

In [66]:
println(match_arr(mat0, mat1))
println(match_arr(mat0, mat2))
println(match_arr(mat0, mat3))
println(match_arr(mat0, mat4))

true
true
true
true


In [62]:
mat3

6×5 SparseMatrixCSC{Int8,Int64} with 6 stored entries:
  [1, 1]  =  1
  [2, 2]  =  1
  [5, 2]  =  1
  [6, 2]  =  1
  [3, 4]  =  1
  [4, 5]  =  1

In [27]:
hexFVarr

6×3 Array{Int64,2}:
 1  2  3
 2  4  3
 4  2  5
 5  2  6
 2  7  6
 2  1  7

In [40]:
if hexFVll<:Array
end

TypeError: TypeError: in <:, expected Type, got Array{Array,1}

In [28]:
methods(array2ll)
# using Base
# Base.delete_method(array2ll)
# Base.delete_method(array2ll)

# 1 method for generic function "array2ll":
[1] array2ll(CVarr::Union{Array{Int64,2}, Array{Int8,2}}) in Main at In[25]:2

In [29]:

println( @elapsed mat = characteristicMatrix_push(aEV, size(V, 2)))

MethodError: MethodError: no method matching characteristicMatrix_push(::Array{Int64,2}, ::Int64)
Closest candidates are:
  characteristicMatrix_push(::Array{Int64,2}) at In[16]:49

In [30]:
mat

543517×272726 SparseMatrixCSC{Int8,Int64} with 1087034 stored entries:
  [38568 ,      1]  =  1
  [285154,      1]  =  1
  [498301,      1]  =  1
  [65258 ,      2]  =  1
  [164256,      2]  =  1
  [193766,      2]  =  1
  [279027,      2]  =  1
  [321165,      3]  =  1
  [347600,      3]  =  1
  [447416,      3]  =  1
  [474415,      3]  =  1
  [525184,      3]  =  1
  ⋮
  [405767, 272724]  =  1
  [441944, 272724]  =  1
  [536136, 272724]  =  1
  [310627, 272725]  =  1
  [333904, 272725]  =  1
  [445369, 272725]  =  1
  [539950, 272725]  =  1
  [49221 , 272726]  =  1
  [80538 , 272726]  =  1
  [176950, 272726]  =  1
  [187932, 272726]  =  1
  [402659, 272726]  =  1

In [31]:
kEV

543517×272726 SparseMatrixCSC{Int8,Int64} with 1087034 stored entries:
  [38568 ,      1]  =  1
  [285154,      1]  =  1
  [498301,      1]  =  1
  [65258 ,      2]  =  1
  [164256,      2]  =  1
  [193766,      2]  =  1
  [279027,      2]  =  1
  [321165,      3]  =  1
  [347600,      3]  =  1
  [447416,      3]  =  1
  [474415,      3]  =  1
  [525184,      3]  =  1
  ⋮
  [405767, 272724]  =  1
  [441944, 272724]  =  1
  [536136, 272724]  =  1
  [310627, 272725]  =  1
  [333904, 272725]  =  1
  [445369, 272725]  =  1
  [539950, 272725]  =  1
  [49221 , 272726]  =  1
  [80538 , 272726]  =  1
  [176950, 272726]  =  1
  [187932, 272726]  =  1
  [402659, 272726]  =  1

In [32]:
typeof(Int64[])

Array{Int64,1}

In [33]:
Array{Int64,1}(undef, 5)

5-element Array{Int64,1}:
 135140976
 135141008
 135141040
 135206800
 135206928

In [34]:
typeof(aEV)

Array{Int64,2}

In [35]:
# LarSurf.Lar.Cells
length(aEV)

1087034

In [36]:
size(aEV)

(543517, 2)

In [37]:
newV = smoothing_EV(newV, kEV, k1)

UndefVarError: UndefVarError: make_full not defined